In [ ]:
import os
os.environ['FP_LOG_LEVEL'] = 'TRACE'

import funkyprompt
from funkyprompt.io.stores import ColumnarDataStore
#the 
from funkyprompt.io import VectorDataStore
from funkyprompt.model import AbstractContentModel, AbstractModel, InstructEmbeddingContentModel
from funkyprompt import FunkyRegistry
d = FunkyRegistry().load()

Model = InstructEmbeddingContentModel.create_model('NewBookChapters')
store= VectorDataStore(Model, description='')


In [ ]:
funkyprompt.agent

In [ ]:
from funkyprompt.model.entity import SchemaOrgVectorEntity
from funkyprompt.io.tools.ingestion import get_page_json_ld_data
url = 'https://www.maangchi.com/recipe/tongbaechu-kimchi'
data = get_page_json_ld_data(url) 
RecipeType = SchemaOrgVectorEntity.create_model_from_schema("Recipe", data)
recipe = RecipeType(**data)
recipe

In [ ]:
vs = VectorDataStore(RecipeType, description='a store for cooking recipes')
vs.add(RecipeType(**data))

In [ ]:
from funkyprompt.io.stores.VectorStoreBase import QueryOptions
opt = QueryOptions(columns=['name','content'])
vs('kimchi', query_options=opt)

In [ ]:
r = vs.as_agent()("tell me exactly how to make Kimchi giving me ingredients in metric units and calling out any key reminders?")
r

In [ ]:
play_response(r)

In [ ]:

# import lance
# lance.write_dataset(df, store._table_uri, schema=store._model.to_arrow_schema(), mode='overwrite')

In [ ]:
 
# #source your data
# url = "https://www.gutenberg.org/files/20748/20748-h/20748-h.htm"
# #create a type for it - it does not need any attributes unless you want to add some
# class FairyTales(InstructEmbeddingContentModel):
#     pass
# from funkyprompt.io.tools.ingestion import iterate_types_from_headed_paragraphs
# data = list(iterate_types_from_headed_paragraphs(url, FairyTales))
Model = InstructEmbeddingContentModel.create_model('FairlyTales')
store = VectorDataStore(Model, description='Classiic Fairy tales - short stories about Snow White, Sinbad, etc')
# store.add(data)
# store.load()

In [ ]:
store._table.search('sinbads retirement location').to_pandas()

In [ ]:
from lancedb.table import _sanitize_data
schema = InstructEmbeddingContentModel.create_model('FairyTalesTester2').to_arrow_schema()
_sanitize_data(data=[{'name':'test', 'content':'test', 'updated_by':None, 'updated_at': None}], schema=schema, metadata=schema.metadata, on_bad_vectors='error', fill_value=0.)

In [ ]:

Model = AbstractModel.create_model(name='NycTaxiData', namespace='default')
store = ColumnarDataStore(Model, description='New York Trips Taxi data are provided for some selected months. People travel in taxis to places like Brooklyn, Manhatten, Queens, Carrol Gardens etc.')
#store.register_store()
#store('trips to carrol gardens')
#store.load()
store.fetch_entities()

In [ ]:
import pandas as pd
df = pd.read_parquet('/Users/sirsh/.funkyprompt/columnar-store/default/NycTaxiData/parts/0/data.parquet')
#for d in ['pick_up_at', 'drop_off_at']:
#    df[d] = pd.to_datetime(df[d])
#df.to_parquet('/Users/sirsh/.funkyprompt/columnar-store/default/NycTaxiData/parts/0/data.parquet')
df['index'] = df['index'].astype(str)
df.rename(columns={'index':'name'}).to_parquet('/Users/sirsh/.funkyprompt/columnar-store/default/NycTaxiData/parts/0/data.parquet')

In [ ]:
ee = store.fetch_entities()

In [ ]:
from funkyprompt.io.stores import AbstractStore
s = AbstractStore.restore_from_data(d.to_dicts()[0],True)
s.function_dict()

In [ ]:
import funkyprompt
funkyprompt.agent(" What was the longest trip durating (in minutes) travelling to New York Airtports in Taxis?")

In [ ]:
import datetime
from funkyprompt.model import NpEncoder
import json

json.dumps([{'longest_trip_duration_to_airport': datetime.timedelta(seconds=31973)}], cls=NpEncoder, default=str)

In [ ]:


Model = AbstractContentModel.create_model(name='zoos', namespace='test')
store = VectorDataStore(Model, description='This one is for storing animal information', register_store=True)
store.register_store()
store

In [ ]:
store.add([
    Model(**{'name':'r1', 'content': 'animals and stuff'}),
    Model(**{'name':'r2', 'content': 'cars and stuff'}),
    Model(**{'name':'r3', 'content': 'space and stuff'}),
    Model(**{'name':'r4', 'content': 'food and stuff'}),
    Model(**{'name':'r5', 'content': 'tech and stuff'}),
    
])
import polars as pl
a = store('look for animals')
a.filter(pl.col('_distance')<0.4)

In [ ]:
query_root = store._table.search('look for animals of any kind such as zebras').limit(4) 
query_root

In [ ]:
store('look for animals fo any kind')

In [ ]:
import funkyprompt
from funkyprompt import agent, describe_function
from funkyprompt.io import add_context
from funkyprompt.ops.examples import get_context
from funkyprompt.ops.entities import AbstractVectorStoreEntry, InstructAbstractVectorStoreEntry, AbstractEntity
from funkyprompt.io.stores import VectorDataStore, list_stores, open_store, get_probe
from funkyprompt.io.tools.ingestion import ingest_pdf, ingest_arrow, simple_scrape_links, ingest_pdf_using_instruct_embedding, SimpleJsonLDSpider, _ingest_web_page, ingest_page_to_model
from funkyprompt import agent
from funkyprompt.ops.utils.inspector import list_functions, load_op
from funkyprompt.io.stores.index import update_function_index, add_cluster_summaries
# list_stores()
# update_function_index()
# store = open_store(**{'type': 'vector-store', 'namespace': 'default', 'name': 'FunctionIndex-instruct'})
# #store = open_store(**{'type': 'vector-store', 'namespace': 'default', 'name': 'PaulGraham'}, embedding_provider='instruct')
# store.load()

store = InstructAbstractVectorStoreEntry.create_model('BookChapters-instruct').as_store()
store

In [ ]:
from pathlib import Path
from openai import OpenAI
import os

def play_response(text, voice='shimmer'):
    """
    temp sample
    """
    client = OpenAI()
    #https://stackoverflow.com/questions/20021457/playing-mp3-song-on-python
    speech_file_path = Path('/tmp') / "speech.mp3"
    response = client.audio.speech.create(
      model="tts-1",
      voice=voice,
      input=text
    )
    response.stream_to_file(speech_file_path)
    os.system(f"afplay {speech_file_path}")
    
    return response

In [ ]:
import openai
openai.__version__

In [ ]:
a = store.as_agent()
a

In [ ]:
a('please summarize 10  great mathamicians in history - why were there contributions important')

In [ ]:
## Inspect functions and Plan execution

In [ ]:
## Ask questions

In [ ]:
## Plot Embeddings